In [18]:
import os
import openai
openai.api_key = ""#自己的openai-key

In [13]:
def request_openai(prompt,model="gpt-3.5-turbo-instruct"):
    response=openai.Completion.create(
              model=model,
              prompt=prompt,
              max_tokens=2000
            )
    return response['choices'][0]['text']


In [17]:
'''
请求GPT3.5生成解释
'''

import json
import jsonlines
from tqdm import tqdm

prompt_file={
            'ET':'事件分类prompt',
            'TRI':'触发词prompt(不包含短语)',
            'ARG':'论元prompt'
        }
data_path="/Users/wyl/Desktop/毕业设计/ee数据集" #数据源
et_prompt_template=open(f'{data_path}/{prompt_file["ET"]}.txt').read()
tri_prompt_template=open(f'{data_path}/{prompt_file["TRI"]}.txt').read()
arg_prompt_template=open(f'{data_path}/{prompt_file["ARG"]}.txt').read()

for prefix in ['train','test']:
    for dataset in ['PHEE','CASIE']:

        f=open(f'{data_path}/{dataset}/{prefix}.json')
        w=open(f'{data_path}/{dataset}/{prefix}1.json','w')
        object = json.load(f)
        print(len(object))

        for item in tqdm(object):
            sentence=item['sentence']
            for event in item['events']:
                event_type=event['type']
                trigger=event['trigger']
                event['EET_explanation']=request_openai(
                    prompt=tri_prompt_template % (sentence,event_type,trigger)
                )
                for argument in event['arguments']:
                    argument_role = argument['role']
                    argument_name=argument['name']
                    argument['EEA_explanation']=request_openai(
                        prompt=arg_prompt_template%(sentence,event_type,argument_role,argument_name)
                    )

        json.dump(object,w)

968


 75%|███████▍  | 724/968 [1:29:46<30:15,  7.44s/it]  


APIConnectionError: Error communicating with OpenAI: HTTPSConnectionPool(host='api.openai.com', port=443): Max retries exceeded with url: /v1/completions (Caused by ProxyError('Cannot connect to proxy.', ConnectionResetError(54, 'Connection reset by peer')))

In [5]:
tokens=\
["Powell", ",", "the", "most", "moderate", "member", "of", "the", "Bush", "cabinet", ",", "said", "he", "fully", "agreed", "with", "the", "president", "'s", "policy", "on", "Iraq", "and", "had", "no", "plans", "to", "leave", "."]
event_trigger_pairs=[{"event_type": "Personnel.End-Position","trigger": "leave"}]

prompt_template=open('/Users/wyl/Desktop/毕业设计/ee数据集/触发词prompt(不包含短语).txt').read()
text=' '.join(tokens)
prompt=prompt_template % (text,json.dumps(event_trigger_pairs))


 The word "leave" triggers a event "End-Position" belongs to "Personnel", the event means Powell has left the Bush cabinet.


In [27]:
with open('/Users/wyl/Desktop/毕业设计/train_explanation.jsonl') as f,open('/Users/wyl/Desktop/毕业设计/train_explanation1.jsonl','w') as g:
    a=f.readlines()[17030:34061]
    for item in a:
        g.write(item)